In [11]:
import torch
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
# lode model and alphabet (our character-level dictionary)
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t30_150M_UR50D")

Using cache found in /home/leobock/.cache/torch/hub/facebookresearch_esm_main


In [23]:
# Load tokenized alphabed
batch_converter = alphabet.get_batch_converter()
# set evaluation mode
model.eval()
# Upload sequences for embedding
df = pd.read_csv('static_data.csv')
data = df.iloc[:,[0] + [-1]]
print(data)

         yORF                                   Protein Sequence
0     YAL001C  MVLTIYPDELVQIVSDKIASNKGKITLNQLWDISGKYFDLSDKKVK...
1     YAL002W  MEQNGLDHDSRSSIDTTINDTQKTFLEFRSYTQLSEKLASSSSYTA...
2     YAL005C  MSKAVGIDLGTTYSCVAHFANDRVDIIANDQGNRTTPSFVAFTDTE...
3     YAL007C  MIKSTIALPSFFIVLILALVNSVAASSSYAPVAISLPAFSKECLYY...
4     YAL008W  MTLAFNMQRLVFRNLNVGKRMFKNVPLWRFNVANKLGKPLTRSVGL...
...       ...                                                ...
4139  YPR187W  MSDYEEAFNDGNENFEDFDVEHFSDEETYEEKPQFKDGETTDANGK...
4140  YPR188C  MDHSESLTFNQLTQDYINKLKDAFQMLDEDEDGLISRGDLTKIYAT...
4141  YPR189W  MSDIKQLLKEAKQELTNRDYEETIEISEKVLKLDPDNYFAHIFLGK...
4142  YPR190C  MDELLGEALSAENQTGESTVESEKLVTPEDVMTISSLEQRTLNPDL...
4143  YPR191W  MLSAARLQFAQGSVRRLTVSARDAPTKISTLAVKVHGGSRYATKDG...

[4144 rows x 2 columns]


In [26]:
data_ = [tuple(row) for row in data.values]

In [27]:
batch_labels, batch_strs, batch_tokens = batch_converter(data_)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[33], return_contacts=True)
token_representations = results["representations"][33]

# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
sequence_representations = []
for i, tokens_len in enumerate(batch_lens):
    sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

# Look at the unsupervised self-attention map contact predictions
for (_, seq), tokens_len, attention_contacts in zip(data, batch_lens, results["contacts"]):
    plt.matshow(attention_contacts[: tokens_len, : tokens_len])
    plt.title(seq)
    plt.show()

RuntimeError: [enforce fail at alloc_cpu.cpp:117] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 52109639680 bytes. Error code 12 (Cannot allocate memory)